In [2]:
from pycocotools import coco
import numpy as np
import skimage.io as io
import matplotlib
import matplotlib.pyplot as plt 
import pylab
import random
import cv2
pylab.rcParams['figure.figsize'] = (8.0, 6.0)

rgbAnnFile='./aauRainSnow-rgb.json'
thermalAnnFile = './aauRainSnow-thermal.json'

rainSnowRgbGt = coco.COCO(rgbAnnFile)
rainSnowThermalGt = coco.COCO(thermalAnnFile)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


## Display a random image with overlayed annotations in both the RGB and thermal domain 


In [ ]:
import os
import matplotlib.pyplot as plt
from skimage import io
import matplotlib

# Ensure the target directories exist
output_dir = "Samples"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(0, 2197):
    chosenImgId = i
    annIds = rainSnowRgbGt.getAnnIds(imgIds=[chosenImgId])
    anns = rainSnowRgbGt.loadAnns(annIds)

    rgbImg = rainSnowRgbGt.loadImgs([chosenImgId])[0]
    thermalImg = rainSnowThermalGt.loadImgs([chosenImgId])[0]
    thermalAnns = rainSnowThermalGt.loadAnns(annIds)

    print('Found ' + str(len(anns)) + ' annotations at image ID ' + str(chosenImgId) + '. Image file: ' + rgbImg['file_name'])

    for ann in anns:
        print('Annotation #' + str(ann['id']) + ': ' + rainSnowRgbGt.loadCats(ann['category_id'])[0]['name'])

    matplotlib.rcParams['interactive'] == False
    print("\nRGB Image")
    I = io.imread('./' + rgbImg['file_name'])
    plt.gcf().clear()
    plt.axis('off')
    plt.imshow(I)
    rainSnowRgbGt.showAnns(anns)
    plt.show()

    # Save the RGB image
    rgb_output_path = os.path.join(output_dir, "rgb-" + str(chosenImgId).zfill(5) + ".png")
    plt.savefig(rgb_output_path)
    print(f"Saved RGB image to {rgb_output_path}")

    print("\nThermal Image")
    # Load thermal annotations
    I = io.imread('./' + thermalImg['file_name'])
    plt.gcf().clear()
    plt.axis('off')
    plt.imshow(I)
    rainSnowThermalGt.showAnns(thermalAnns)
    plt.show()

    # Save the thermal image
    thermal_output_path = os.path.join(output_dir, "thermal-" + str(chosenImgId).zfill(5) + ".png")
    plt.savefig(thermal_output_path)
    print(f"Saved Thermal image to {thermal_output_path}")


## Register an annotation in RGB to the thermal domain

In [4]:
import aauRainSnowUtility

chosenImgId = random.randint(0, 2197)
annIds = rainSnowRgbGt.getAnnIds(imgIds=[chosenImgId])
anns = rainSnowRgbGt.loadAnns(annIds)
rgbImg = rainSnowRgbGt.loadImgs([chosenImgId])[0]


if len(anns) > 0:
    chosenAnnId = random.randint(0, len(anns)-1)
    rgbAnn = anns[chosenAnnId]
    
    thermalSegmentation = []
    for segmentation in rgbAnn['segmentation']:
        thermalCoords = aauRainSnowUtility.registerRgbPointsToThermal(segmentation, rgbImg['file_name'])
        
        
        thermalSegmentation.append(thermalCoords)
        
        print('RGB coordinates for annotation ID ' + str(rgbAnn['id']) +':\n' + str(np.reshape(segmentation, (-1, 2))))
        print('Thermal coordinates:\n' + str(thermalCoords.reshape([-1, 2])))    
    
else:
    print("No annotations found for image ID " + str(chosenImgId) + ", try again")



Reading calibration file at: Egensevej\Egensevej-2-calib.yml
RGB coordinates for annotation ID 588:
[[412 155]
 [411 156]
 [411 162]
 [412 163]
 [419 163]
 [420 162]
 [420 157]
 [419 156]
 [417 156]
 [416 155]]
Thermal coordinates:
[[430.22998 128.2242 ]]


## Register an annotation in thermal to the RGB domain

In [6]:
import aauRainSnowUtility

chosenImgId = random.randint(0, 2197)
annIds = rainSnowThermalGt.getAnnIds(imgIds=[chosenImgId])
anns = rainSnowThermalGt.loadAnns(annIds)
thermalImg = rainSnowThermalGt.loadImgs([chosenImgId])[0]


if len(anns) > 0:
    chosenAnnId = random.randint(0, len(anns)-1)
    thermalAnn = anns[chosenAnnId]
    
    rgbSegmentation = []
    for segmentation in thermalAnn['segmentation']:
        rgbCoords = aauRainSnowUtility.registerThermalPointsToRgb(segmentation, thermalImg['file_name'])
        
        
        rgbSegmentation.append(rgbCoords)
        
        print('Thermal coordinates for annotation ID ' + str(thermalAnn['id']) +':\n' + str(np.reshape(segmentation, (-1, 2))))
        print('RGB coordinates:\n' + str(rgbCoords.reshape([-1, 2])))    
    
else:
    print("No annotations found for image ID " + str(chosenImgId) + ", try again")

Reading calibration file at: Egensevej\Egensevej-4-calib.yml
Thermal coordinates for annotation ID 1196:
[[421.58728027 160.44464111]
 [420.56378174 161.49856567]
 [420.79312134 173.48254395]
 [421.85726929 174.60447693]
 [433.30441284 174.96685791]
 [434.32272339 173.91386414]
 [434.10543823 161.96305847]
 [433.04650879 160.8401947 ]]
RGB coordinates:
[[403. 185.]]
